# Semantische Kernel

In dit codevoorbeeld gebruik je het [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI-framework om een eenvoudige agent te maken.

Het doel van dit voorbeeld is om je de stappen te laten zien die we later in de aanvullende codevoorbeelden zullen gebruiken bij het implementeren van de verschillende agentpatronen.


## Importeer de benodigde Python-pakketten


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## De Client Maken

In dit voorbeeld gebruiken we [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) voor toegang tot de LLM.

De `ai_model_id` is gedefinieerd als `gpt-4o-mini`. Probeer het model te wijzigen naar een ander model dat beschikbaar is in de GitHub Models-marktplaats om de verschillende resultaten te zien.

Om de `Azure Inference SDK` te gebruiken die wordt gebruikt voor de `base_url` voor GitHub Models, zullen we de `OpenAIChatCompletion` connector binnen Semantic Kernel gebruiken. Er zijn ook andere [beschikbare connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) om Semantic Kernel te gebruiken voor andere modelproviders.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## De Agent maken

Hier maken we de Agent genaamd `TravelAgent`.

Voor dit voorbeeld gebruiken we zeer eenvoudige instructies. Je kunt deze instructies aanpassen om te zien hoe de agent anders reageert.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## De Agent Uitvoeren

Nu kunnen we de Agent uitvoeren door een thread van het type `ChatHistoryAgentThread` te definiëren. Eventuele benodigde systeemberichten worden aan het `messages` keyword-argument van de agent's invoke_stream meegegeven.

Zodra deze zijn gedefinieerd, maken we een `user_inputs` aan die de boodschap bevat die de gebruiker naar de agent stuurt. In dit geval hebben we dit bericht ingesteld op `Plan me a sunny vacation`.

Voel je vrij om dit bericht aan te passen om te zien hoe de agent anders reageert.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
